# Synapse Analyticsにアクセスしてデータの読み書きを行う

**参考資料**
- [DatabricksとAzure Synapse Analyticsの連携 \- Qiita](https://qiita.com/taka_yayoi/items/7d830a6a273dadd94c2a)
- [チュートリアル:Azure Synapse Analytics の使用を開始する \- Azure Synapse Analytics \| Microsoft Docs](https://docs.microsoft.com/ja-jp/azure/synapse-analytics/get-started)

## Synapse Workspaceの作成

ここではデモの目的でダミーのワークスペースを作成しています。既存のワークスペースが存在する場合には、そちらを参照してください。

ここでは以下の設定を行なっています。

|項目|設定値|
|:--|:--|
|workspace名|taka-workspace|
|ストレージアカウント名|takaaccount|
|ストレージコンテナー名|users|

![](https://sajpstorage.blob.core.windows.net/demo20210509-synapse/create_workspace.png)

## 専用のSQLプールを作成

ここでは「mysqlpool」と言うSQLプールを作成しています。

![](https://sajpstorage.blob.core.windows.net/demo20210509-synapse/create_sql_pool.png)

## Synapseワークスペースの設定の確認

### ファイアウォール設定の確認

DatabricksからSynapseにアクセスできるようにするためには、Synapseのファイアウォールの設定で「Azure サービスおよびリソースに、このワークスペースへのアクセスを許可する」がオンになっていることを確認します。

![](https://sajpstorage.blob.core.windows.net/demo20210509-synapse/firewall.png)

### SQL 管理ユーザー名の確認

ワークスペースの概要に表示される「SQL 管理ユーザー名」をメモしておいてください。また、パスワードもご確認ください。これらはSynapse接続時に指定する必要があります。

![](https://sajpstorage.blob.core.windows.net/demo20210509-synapse/sqladmin.png)

### SQLプールのマスターパスワードが作成されていることを確認

DatabricksからSQLプールにアクセスして操作を行う場合には、当該SQLプールでマスターパスワードが作成されている必要があります。作成されていない場合には、当該SQLプールで以下のSQLを実行してください。
<br>
- [Create a Database Master Key \- SQL Server \| Microsoft Docs](https://docs.microsoft.com/en-us/sql/relational-databases/security/encryption/create-a-database-master-key?view=sql-server-ver15)

![](https://sajpstorage.blob.core.windows.net/demo20210509-synapse/master_password.png)

## ストレージアカウントのアクセスキーを設定

ストレージアカウントのアクセスキーは、ホーム > ストレージアカウントで「キーの表示」をクリックし、表示されるKey1を指定します。

![](https://sajpstorage.blob.core.windows.net/demo20210509-synapse/storage_account_access_key.png)

以下ではアクセスキーやパスワードを平文で記載していますが、本運用の際にはシークレットの活用をご検討ください。
<br><br>
**参考資料**
- [Databricksにおけるシークレットの管理 \- Qiita](https://qiita.com/taka_yayoi/items/338ef0c5394fe4eb87c0)

In [0]:
storage_account_key = "<<ストレージアカウントのアクセスキー>>"

spark.conf.set("fs.azure.account.key.takaaccount.blob.core.windows.net", storage_account_key)

## JDBCユーザー名とパスワードで接続しデータを読み込む
<br>
- `hostname` Workspace SQL endpointを指定
- `database` SQL pool名を指定
- `dbuser` SQL 管理ユーザー名を`ユーザー名@ワークスペース名`の形式で指定
- `dbpassword` = SQL 管理ユーザーのパスワードを指定
- `storage_account` Workspace作成時に指定したストレージアカウント名を指定
- `container_name` = Workspace作成時に指定したストレージコンテナー名を指定
- `table_name` = 読み取るテーブル名を指定

In [0]:
hostname = "taka-workspace.sql.azuresynapse.net"
database = "mysqlpool"
dbuser = "sqladminuser@taka-workspace"
dbpassword = "<<SQL管理ユーザーのパスワード>>"
storage_account = "takaaccount"
container_name = "users"
table_name = "NYCTaxiTripSmall"

# Get some data from an Azure Synapse table.
df = spark.read \
  .format("com.databricks.spark.sqldw") \
  .option("url", "jdbc:sqlserver://{0}:1433;database={1};user={2};password={3};trustServerCertificate=false;hostNameInCertificate=*.sql.azuresynapse.net;loginTimeout=30;".format(hostname, database, dbuser, dbpassword)) \
  .option("tempDir", "wasbs://{0}@{1}.blob.core.windows.net/tempdir".format(container_name, storage_account)) \
  .option("forwardSparkAzureStorageCredentials", "true") \
  .option("dbTable", table_name) \
  .load()

In [0]:
display(df)

DateID,MedallionID,HackneyLicenseID,PickupTimeID,DropoffTimeID,PickupGeographyID,DropoffGeographyID,PickupLatitude,PickupLongitude,PickupLatLong,DropoffLatitude,DropoffLongitude,DropoffLatLong,PassengerCount,TripDurationSeconds,TripDistanceMiles,PaymentType,FareAmount,SurchargeAmount,TaxAmount,TipAmount,TollsAmount,TotalAmount
20131230,484,28116,62340,62820,156505,21917,40.793,-73.975,"40.793,-73.975",40.7817,-73.954,"40.7817,-73.954",1,480,2.11,CRD,9.0000,1.0000,0.5000,2.0000,0.0000,12.5000
20131229,9985,14769,78822,79139,213112,66499,40.796,-73.9722,"40.796,-73.9722",40.782,-73.982,"40.782,-73.982",1,317,1.1,CRD,6.5000,0.5000,0.5000,1.8700,0.0000,9.3700
20131230,1382,23605,44520,44880,265190,137087,40.7829,-73.9531,"40.7829,-73.9531",40.7726,-73.9607,"40.7726,-73.9607",1,360,0.84,CSH,5.5000,0.0000,0.5000,0.0000,0.0000,6.0000
20131229,4305,18190,40816,41527,41374,88048,40.7019,-74.0097,"40.7019,-74.0097",40.7152,-74.016,"40.7152,-74.016",1,711,1.8,CSH,9.5000,0.0000,0.5000,0.0000,0.0000,10.0000
20131230,11435,9324,44640,44760,1841,281081,40.7591,-73.9707,"40.7591,-73.9707",40.7524,-73.9755,"40.7524,-73.9755",1,120,0.54,CSH,4.0000,0.0000,0.5000,0.0000,0.0000,4.5000
20131229,13163,28329,73864,74416,302273,257931,40.7152,-74.0147,"40.7152,-74.0147",40.7061,-74.0085,"40.7061,-74.0085",2,552,1.4,CSH,7.5000,0.5000,0.5000,0.0000,0.0000,8.5000
20131230,4240,25554,26932,27277,259445,97730,40.7774,-73.9789,"40.7774,-73.9789",40.764,-73.9733,"40.764,-73.9733",1,344,1.3,CRD,7.0000,0.0000,0.5000,1.0000,0.0000,8.5000
20131229,82,1805,59637,60002,98261,226503,40.7206,-74.0086,"40.7206,-74.0086",40.7058,-74.018,"40.7058,-74.018",2,365,1.4,CSH,7.0000,0.0000,0.5000,0.0000,0.0000,7.5000
20131230,6311,19413,33211,34089,282835,154558,40.7437,-73.9738,"40.7437,-73.9738",40.7401,-74.0012,"40.7401,-74.0012",1,877,2.1,CRD,11.0000,0.0000,0.5000,2.0000,0.0000,13.5000
20131229,4272,15795,75000,75480,253129,270907,40.7523,-73.9935,"40.7523,-73.9935",40.7343,-74.003,"40.7343,-74.003",3,480,1.8,CSH,8.5000,0.5000,0.5000,0.0000,0.0000,9.5000


## Synapse上にテーブルを作成する

ここでは、Databricksデータセットに格納されているダイアモンドのデータセットをSynapse上に作成します。

In [0]:
dataFrame = "/databricks-datasets/Rdatasets/data-001/csv/ggplot2/diamonds.csv"
diamonds = spark.read.format("csv").option("header","true")\
  .option("inferSchema", "true").load(dataFrame)

In [0]:
display(diamonds)

_c0,carat,cut,color,clarity,depth,table,price,x,y,z
1,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
2,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
3,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
4,0.29,Premium,I,VS2,62.4,58.0,334,4.2,4.23,2.63
5,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75
6,0.24,Very Good,J,VVS2,62.8,57.0,336,3.94,3.96,2.48
7,0.24,Very Good,I,VVS1,62.3,57.0,336,3.95,3.98,2.47
8,0.26,Very Good,H,SI1,61.9,55.0,337,4.07,4.11,2.53
9,0.22,Fair,E,VS2,65.1,61.0,337,3.87,3.78,2.49
10,0.23,Very Good,H,VS1,59.4,61.0,338,4.0,4.05,2.39


[書き込みのセマンティクス](https://qiita.com/taka_yayoi/items/7d830a6a273dadd94c2a#%E6%9B%B8%E3%81%8D%E8%BE%BC%E3%81%BF%E3%81%AE%E3%82%BB%E3%83%9E%E3%83%B3%E3%83%86%E3%82%A3%E3%82%AF%E3%82%B9)

> Azure SynapseコネクターはCOPY文をサポートしています。COPY文は、外部テーブルを作成することなしにデータロードが可能で、データロードに必要な権限が少なくてすみ、Azure Synapseに対して高速なデータ投入を可能とする便利な方法を提供します。

In [0]:
# COPY文の使用を強制
spark.conf.set("spark.databricks.sqldw.writeSemantics", "copy")

# 書き込み先のテーブル名
write_table_name = "diamonds"

diamonds.write \
  .format("com.databricks.spark.sqldw") \
  .option("url", "jdbc:sqlserver://{0}:1433;database={1};user={2};password={3};trustServerCertificate=false;hostNameInCertificate=*.sql.azuresynapse.net;loginTimeout=30;".format(hostname, database, dbuser, dbpassword)) \
  .option("tempDir", "wasbs://{0}@{1}.blob.core.windows.net/tempdir".format(container_name, storage_account)) \
  .option("forwardSparkAzureStorageCredentials", "true") \
  .option("dbTable", write_table_name) \
  .save()

Synapse Analytics側でデータを確認します。

![](https://sajpstorage.blob.core.windows.net/demo20210509-synapse/diamonds.png)

# END